This notebook illustrate how to query the EFD using the `lsst_efd_client` library. 

The queries bellow are mainly inspired in some of the queries we normally do. Documentation about how to install and setup the library can be seen here https://efd-client.lsst.io/#

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample

%matplotlib inline

For this next line to work you need to setup the `notebook_auth.yaml` file as instructed here https://efd-client.lsst.io/getting-started.html#authentication-configuration

In [ ]:
efd_client = EfdClient('summit_efd')

Defining a timestamp to query data.

In [ ]:
t1 = Time("2020-01-23T00:21:00", format='isot', scale='tai')
t2 = Time("2020-01-23T06:59:00", format='isot', scale='tai')

Query simple entry in the database.

In [ ]:
hexapod = await efd_client.select_time_series("lsst.sal.ATHexapod.command_moveToPosition",
                                              ["x", "y", "z", "u", "v", "w"], t1, t2)

Query packed entries in the database.

Packed entries are created when a topic has an array of elements. Since influxdb does not have array types, the producer breaks arrays with `n` elements into `n` values in the datase. The `efd_client` libraries can pack these data types back and merge them into a unique table. 

In [ ]:
azel = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders",
                                                  ["elevationCalculatedAngle", "azimuthCalculatedAngle"], t1, t2)

Make a user-defined query to the database. 

These are usefull when you want to match a specific value or values in the database instead of quering for time windows. 

In [ ]:
my_query = 'SELECT "imageName" FROM "efd"."autogen"."lsst.sal.GenericCamera.logevent_endReadout" where' \
           '"imageName" = \'1576286733-HD32313-0-1\''

In [ ]:
eron = await efd_client.influx_client.query(my_query)